In [ ]:
%%capture
%pip install torch
%pip install transformers
%pip install accelerate
%pip install safetensors
%pip install pytorch_memlab
%pip install huggingface_hub

# **LOOK**

Memory Profiler

In [ ]:
!pip install -U memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## ***-------------------------------[1]-------------------------------***

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM
def func():
  #config = AutoConfig.from_pretrained("decapoda-research/llama-65b-hf", low_cpu_mem_usage=True)
  config = AutoConfig.from_pretrained("pszemraj/flan-t5-large-grammar-synthesis")
  with init_empty_weights():
    model = AutoModelForSeq2SeqLM.from_config(config) 
  device_map = infer_auto_device_map(model)


In [ ]:
import cProfile
import re
cProfile.run('model = AutoModelForSeq2SeqLM.from_config(config)')

         135910 function calls (113779 primitive calls) in 18.563 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   18.563   18.563 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:1436(info)
        1    0.000    0.000    0.000    0.000 __init__.py:1689(isEnabledFor)
        1    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
       38    0.000    0.000    0.000    0.000 __init__.py:97(_xla_gc_callback)
      436    0.000    0.000    0.000    0.000 _jit_internal.py:1082(is_scripting)
        2    0.000    0.000    0.000    0.000 abc.py:117(__instancecheck__)
       48    0.000    0.000    0.002    0.000 activations.py:167(__getitem__)
       48    0.000    0.000    0.001    0.000 activations.py:67(__init__)
        1    0.000    0.000    0.000    0.000 auto_factory.py:359(_get_model_class)
        1    0.000    0.000   18.563   18.563 auto_factory.py:390(fr

In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [ ]:
device_map

{'': 'disk'}

In [ ]:
model.state_dict()

OrderedDict([('transformer.word_embeddings.weight',
              tensor(..., device='meta', size=(250880, 14336))),
             ('transformer.word_embeddings_layernorm.weight',
              tensor(..., device='meta', size=(14336,))),
             ('transformer.word_embeddings_layernorm.bias',
              tensor(..., device='meta', size=(14336,))),
             ('transformer.h.0.input_layernorm.weight',
              tensor(..., device='meta', size=(14336,))),
             ('transformer.h.0.input_layernorm.bias',
              tensor(..., device='meta', size=(14336,))),
             ('transformer.h.0.self_attention.query_key_value.weight',
              tensor(..., device='meta', size=(43008, 14336))),
             ('transformer.h.0.self_attention.query_key_value.bias',
              tensor(..., device='meta', size=(43008,))),
             ('transformer.h.0.self_attention.dense.weight',
              tensor(..., device='meta', size=(14336, 14336))),
             ('transformer.h.0.s

## ***-------------------------------[2]-------------------------------***

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM

config = AutoConfig.from_pretrained("facebook/opt-6.7b", low_cpu_mem_usage=True)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)
    
device_map = infer_auto_device_map(model)

In [ ]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=4096, out_features=16384, bias=True)
          (fc2): Linear(in_features=16384, out_features=4096, bias=True)
          (final_layer_norm): LayerNorm((4

In [ ]:
device_map

{'model.decoder.embed_tokens': 0,
 'model.decoder.embed_positions': 0,
 'model.decoder.final_layer_norm': 0,
 'model.decoder.layers.0': 0,
 'model.decoder.layers.1': 0,
 'model.decoder.layers.2': 0,
 'model.decoder.layers.3': 0,
 'model.decoder.layers.4': 0,
 'model.decoder.layers.5': 0,
 'model.decoder.layers.6': 0,
 'model.decoder.layers.7': 0,
 'model.decoder.layers.8': 0,
 'model.decoder.layers.9': 0,
 'model.decoder.layers.10': 0,
 'model.decoder.layers.11': 0,
 'model.decoder.layers.12': 0,
 'model.decoder.layers.13': 0,
 'model.decoder.layers.14': 0,
 'model.decoder.layers.15': 0,
 'model.decoder.layers.16.self_attn': 0,
 'model.decoder.layers.16.activation_fn': 0,
 'model.decoder.layers.16.self_attn_layer_norm': 0,
 'model.decoder.layers.16.fc1': 0,
 'model.decoder.layers.16.fc2': 'cpu',
 'model.decoder.layers.16.final_layer_norm': 'cpu',
 'model.decoder.layers.17': 'cpu',
 'model.decoder.layers.18': 'cpu',
 'model.decoder.layers.19': 'cpu',
 'model.decoder.layers.20': 'cpu',
 

In [ ]:
model.state_dict()

OrderedDict([('model.decoder.embed_tokens.weight',
              tensor(..., device='meta', size=(50272, 4096))),
             ('model.decoder.embed_positions.weight',
              tensor(..., device='meta', size=(2050, 4096))),
             ('model.decoder.final_layer_norm.weight',
              tensor(..., device='meta', size=(4096,))),
             ('model.decoder.final_layer_norm.bias',
              tensor(..., device='meta', size=(4096,))),
             ('model.decoder.layers.0.self_attn.k_proj.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('model.decoder.layers.0.self_attn.k_proj.bias',
              tensor(..., device='meta', size=(4096,))),
             ('model.decoder.layers.0.self_attn.v_proj.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('model.decoder.layers.0.self_attn.v_proj.bias',
              tensor(..., device='meta', size=(4096,))),
             ('model.decoder.layers.0.self_attn.q_proj.wei

## ***-------------------------------[3]-------------------------------***

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("facebook/opt-13b", low_cpu_mem_usage=True)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

device_map = infer_auto_device_map(model)


In [ ]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 5120, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 5120)
      (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (v_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (out_proj): Linear(in_features=5120, out_features=5120, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=5120, out_features=20480, bias=True)
          (fc2): Linear(in_features=20480, out_features=5120, bias=True)
          (final_layer_norm): LayerNorm((5

In [ ]:
device_map

{'model.decoder.embed_tokens': 0,
 'model.decoder.embed_positions': 0,
 'model.decoder.final_layer_norm': 0,
 'model.decoder.layers.0': 0,
 'model.decoder.layers.1': 0,
 'model.decoder.layers.2': 0,
 'model.decoder.layers.3': 0,
 'model.decoder.layers.4': 0,
 'model.decoder.layers.5': 0,
 'model.decoder.layers.6': 0,
 'model.decoder.layers.7': 0,
 'model.decoder.layers.8': 0,
 'model.decoder.layers.9': 0,
 'model.decoder.layers.10.self_attn': 0,
 'model.decoder.layers.10.activation_fn': 0,
 'model.decoder.layers.10.self_attn_layer_norm': 0,
 'model.decoder.layers.10.fc1': 'cpu',
 'model.decoder.layers.10.fc2': 'cpu',
 'model.decoder.layers.10.final_layer_norm': 'cpu',
 'model.decoder.layers.11': 'cpu',
 'model.decoder.layers.12': 'cpu',
 'model.decoder.layers.13': 'cpu',
 'model.decoder.layers.14': 'cpu',
 'model.decoder.layers.15': 'cpu',
 'model.decoder.layers.16': 'cpu',
 'model.decoder.layers.17.self_attn': 'cpu',
 'model.decoder.layers.17.activation_fn': 'cpu',
 'model.decoder.lay

In [ ]:
model.state_dict

<bound method Module.state_dict of OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 5120, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 5120)
      (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (v_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
            (out_proj): Linear(in_features=5120, out_features=5120, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=5120, out_features=20480, bias=True)
          (fc2): Linear(in_features=20480, out_features=5120, bias=True)
       

In [ ]:
model.config.architectures

['OPTForCausalLM']

In [ ]:
from transformers import AutoModelForSeq2SeqLM

#t0pp = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", low_cpu_mem_usage=True)

## ***-------------------------------[4]-------------------------------***

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM

config = AutoConfig.from_pretrained("google/flan-t5-xxl", low_cpu_mem_usage=True)
with init_empty_weights():
    model = AutoModelForSeq2SeqLM.from_config(config)

device_map = infer_auto_device_map(model)



In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 4096)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 4096)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=4096, out_features=4096, bias=False)
              (k): Linear(in_features=4096, out_features=4096, bias=False)
              (v): Linear(in_features=4096, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=4096, bias=False)
              (relative_attention_bias): Embedding(32, 64)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=4096, out_features=10240, bias=False)
              (wi_1): Linear(in_features=4096, out_features=10240, bias=False)
     

In [ ]:
device_map

{'shared': 0,
 'decoder.embed_tokens': 0,
 'encoder.embed_tokens': 0,
 'encoder.block.0': 0,
 'encoder.block.1': 0,
 'encoder.block.2': 0,
 'encoder.block.3': 0,
 'encoder.block.4': 0,
 'encoder.block.5': 0,
 'encoder.block.6': 0,
 'encoder.block.7': 0,
 'encoder.block.8': 0,
 'encoder.block.9': 0,
 'encoder.block.10': 0,
 'encoder.block.11': 0,
 'encoder.block.12': 0,
 'encoder.block.13': 0,
 'encoder.block.14': 0,
 'encoder.block.15': 0,
 'encoder.block.16': 0,
 'encoder.block.17.layer.0': 0,
 'encoder.block.17.layer.1.DenseReluDense.wi_0': 0,
 'encoder.block.17.layer.1.DenseReluDense.wi_1': 'cpu',
 'encoder.block.17.layer.1.DenseReluDense.wo': 'cpu',
 'encoder.block.17.layer.1.DenseReluDense.dropout': 'cpu',
 'encoder.block.17.layer.1.DenseReluDense.act': 'cpu',
 'encoder.block.17.layer.1.layer_norm': 'cpu',
 'encoder.block.17.layer.1.dropout': 'cpu',
 'encoder.block.18': 'cpu',
 'encoder.block.19': 'cpu',
 'encoder.block.20': 'cpu',
 'encoder.block.21': 'cpu',
 'encoder.block.22': 

In [ ]:
model.state_dict()

OrderedDict([('shared.weight', tensor(..., device='meta', size=(32128, 4096))),
             ('encoder.embed_tokens.weight',
              tensor(..., device='meta', size=(32128, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.q.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.k.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.v.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.o.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight',
              tensor(..., device='meta', size=(32, 64))),
             ('encoder.block.0.layer.0.layer_norm.weight',
              tensor(..., device='meta', size=(4096,))),
             ('encoder.block.0.layer.1.DenseRelu

In [ ]:
#model1.load_state_dict(model2.state_dict())

## ***-------------------------------[5]-------------------------------***

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM

config = AutoConfig.from_pretrained("bigscience/T0pp", low_cpu_mem_usage=True)

with init_empty_weights():
    model = AutoModelForSeq2SeqLM.from_config(config)

device_map = infer_auto_device_map(model)



In [ ]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 4096)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 4096)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=4096, out_features=4096, bias=False)
              (k): Linear(in_features=4096, out_features=4096, bias=False)
              (v): Linear(in_features=4096, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=4096, bias=False)
              (relative_attention_bias): Embedding(32, 64)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=4096, out_features=10240, bias=False)
              (wi_1): Linear(in_features=4096, out_features=10240, bias=False)
     

In [ ]:
device_map

{'shared': 0,
 'decoder.embed_tokens': 0,
 'encoder.embed_tokens': 0,
 'encoder.block.0': 0,
 'encoder.block.1': 0,
 'encoder.block.2': 0,
 'encoder.block.3': 0,
 'encoder.block.4': 0,
 'encoder.block.5': 0,
 'encoder.block.6': 0,
 'encoder.block.7': 0,
 'encoder.block.8': 0,
 'encoder.block.9': 0,
 'encoder.block.10': 0,
 'encoder.block.11': 0,
 'encoder.block.12': 0,
 'encoder.block.13': 0,
 'encoder.block.14': 0,
 'encoder.block.15': 0,
 'encoder.block.16': 0,
 'encoder.block.17': 0,
 'encoder.block.18.layer.0.SelfAttention.q': 0,
 'encoder.block.18.layer.0.SelfAttention.k': 0,
 'encoder.block.18.layer.0.SelfAttention.v': 0,
 'encoder.block.18.layer.0.SelfAttention.o': 'cpu',
 'encoder.block.18.layer.0.layer_norm': 'cpu',
 'encoder.block.18.layer.0.dropout': 'cpu',
 'encoder.block.18.layer.1': 'cpu',
 'encoder.block.19': 'cpu',
 'encoder.block.20': 'cpu',
 'encoder.block.21': 'cpu',
 'encoder.block.22': 'cpu',
 'encoder.block.23': 'cpu',
 'encoder.final_layer_norm': 'cpu',
 'encoder

In [ ]:
model.state_dict()

OrderedDict([('shared.weight', tensor(..., device='meta', size=(32128, 4096))),
             ('encoder.embed_tokens.weight',
              tensor(..., device='meta', size=(32128, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.q.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.k.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.v.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.o.weight',
              tensor(..., device='meta', size=(4096, 4096))),
             ('encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight',
              tensor(..., device='meta', size=(32, 64))),
             ('encoder.block.0.layer.0.layer_norm.weight',
              tensor(..., device='meta', size=(4096,))),
             ('encoder.block.0.layer.1.DenseRelu

# **LLM**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
#This code load all the model weights
"""
tokenizer = AutoTokenizer.from_pretrained("cerebras/Cerebras-GPT-13B")
model = AutoModelForCausalLM.from_pretrained("cerebras/Cerebras-GPT-13B")

text = "Generative AI is "
"""

In [ ]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoModelForSeq2SeqLM

config = AutoConfig.from_pretrained("cerebras/Cerebras-GPT-13B", low_cpu_mem_usage=True)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

device_map = infer_auto_device_map(model)

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 5120)
    (wpe): Embedding(2048, 5120)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): GELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): D

In [ ]:
device_map

{'transformer.wte': 'cpu',
 'transformer.wpe': 'cpu',
 'transformer.drop': 'cpu',
 'transformer.h.0': 'cpu',
 'transformer.h.1': 'cpu',
 'transformer.h.2': 'cpu',
 'transformer.h.3': 'cpu',
 'transformer.h.4': 'cpu',
 'transformer.h.5': 'cpu',
 'transformer.h.6': 'cpu',
 'transformer.h.7.ln_1': 'cpu',
 'transformer.h.7.attn': 'cpu',
 'transformer.h.7.ln_2': 'cpu',
 'transformer.h.8': 'disk',
 'transformer.h.9': 'disk',
 'transformer.h.10': 'disk',
 'transformer.h.11': 'disk',
 'transformer.h.12': 'disk',
 'transformer.h.13': 'disk',
 'transformer.h.14': 'disk',
 'transformer.h.15': 'disk',
 'transformer.h.16': 'disk',
 'transformer.h.17': 'disk',
 'transformer.h.18': 'disk',
 'transformer.h.19': 'disk',
 'transformer.h.20': 'disk',
 'transformer.h.21': 'disk',
 'transformer.h.22': 'disk',
 'transformer.h.23': 'disk',
 'transformer.h.24': 'disk',
 'transformer.h.25': 'disk',
 'transformer.h.26': 'disk',
 'transformer.h.27': 'disk',
 'transformer.h.28': 'disk',
 'transformer.h.29': 'disk

In [ ]:
import torch
from pytorch_memlab import profile, set_target_gpu
@profile
def func():
    net1 = torch.nn.Linear(1024, 1024)
    set_target_gpu(1)
    net2 = torch.nn.Linear(1024, 1024)
    set_target_gpu(0)
    net3 = torch.nn.Linear(1024, 1024)

func()

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="torch.distributed.reduce_op is deprecated")

# **Various**

In [ ]:
from transformers import AutoModelForSeq2SeqLM

#t0pp = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", low_cpu_mem_usage=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

#t0pp = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map="auto")
#t0pp.hf_device_map

In [ ]:
import accelerate
from accelerate import accelerator 

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("model.safetensors", framework="pt", device=0) as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k)

In [ ]:
import os
import datetime
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import torch

In [ ]:
sf_filename = hf_hub_download("gpt2", filename="model.safetensors")


In [ ]:
sf_filename

'/root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/model.safetensors'

In [ ]:
pt_filename = hf_hub_download("gpt2", filename="pytorch_model.bin")

In [ ]:
pt_filename

'/root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/pytorch_model.bin'

In [ ]:
start_st = datetime.datetime.now()
weights = load_file(sf_filename, device="cpu")
load_time_st = datetime.datetime.now() - start_st
print(f"Loaded safetensors {load_time_st}")

start_pt = datetime.datetime.now()
weights = torch.load(pt_filename, map_location="cpu")
load_time_pt = datetime.datetime.now() - start_pt
print(f"Loaded pytorch {load_time_pt}")

print(f"on CPU, safetensors is faster than pytorch by: {load_time_pt/load_time_st:.1f} X")

Loaded safetensors 0:00:00.055665
Loaded pytorch 0:00:00.306339
on CPU, safetensors is faster than pytorch by: 5.5 X


In [ ]:
# This is required because this feature hasn't been fully verified yet, but 
# it's been tested on many different environments
os.environ["SAFETENSORS_FAST_GPU"] = "1"

# CUDA startup out of the measurement
torch.zeros((2, 2)).cuda()

start_st = datetime.datetime.now()
weights = load_file(sf_filename, device="cuda:0")
load_time_st = datetime.datetime.now() - start_st
print(f"Loaded safetensors {load_time_st}")

start_pt = datetime.datetime.now()
weights = torch.load(pt_filename, map_location="cuda:0")
load_time_pt = datetime.datetime.now() - start_pt
print(f"Loaded pytorch {load_time_pt}")

print(f"on GPU, safetensors is faster than pytorch by: {load_time_pt/load_time_st:.1f} X")

Loaded safetensors 0:00:00.174463
Loaded pytorch 0:00:00.316121
on GPU, safetensors is faster than pytorch by: 1.8 X


In [ ]:
import torch
from transformers import pipeline

# This works on a base Colab instance.
# Pick a larger checkpoint if you have time to wait and enough disk space!
checkpoint = "facebook/opt-6.7b"
generator = pipeline("text-generation", model=checkpoint, device_map="auto", torch_dtype=torch.float16)

# Perform inference
generator("More and more large language models are opensourced so Hugging Face has")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[{'generated_text': 'More and more large language models are opensourced so Hugging Face has decided to open source their own'}]